In [196]:
#import Dependencies
import pandas as pd
import os
import matplotlib.pyplot as plt
pd.reset_option("all")



: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [197]:
#create a varialbe for the file
file = "purchase_data.csv"

In [210]:
#import and view the first 5 lines of the data file
purchase_data_df = pd.read_csv(file)


In [199]:
#get player count
#break to a new df with only the data we need
player_count_df = purchase_data_df.copy()["SN"]
#count unique Screen Names
player_count = player_count_df.nunique()


In [200]:
#Purchasting Analysis(Total)
#Number of Unique Items
number_items_df = purchase_data_df.copy()["Item Name"]
number_items = number_items_df.nunique()

#Average Purchase Price
average_purcahse_df = purchase_data_df.copy()["Price"]
average_purchase = average_purcahse_df.mean()

#Total Number of Purchases

total_num_purchases_df = purchase_data_df.copy()["Purchase ID"]
total_num_purchases = total_num_purchases_df.count()

#Total Revenue
total_rev_df = purchase_data_df.copy()["Price"]
total_rev = total_rev_df.sum()



In [201]:
#Gender Demographics
#Percentage and Count of Male Players
male_count = purchase_data_df.copy().loc[purchase_data_df.copy()["Gender"] == 'Male',:].count()
male_count = male_count["Gender"]
male_percent = male_count/player_count
male_percent = "{:.2%}".format(male_percent)

#Percentage and Count of Female Players
female_count = purchase_data_df.copy().loc[purchase_data_df.copy()["Gender"] == 'Female',:].count()
female_count = female_count["Gender"]
female_percent = female_count/player_count
female_percent = "{:.2%}".format(female_percent)

#Percentage and Count of Other / Non-Disclosed
other_count = purchase_data_df.copy().loc[purchase_data_df.copy()["Gender"] == 'Other / Non-Disclosed',:].count()
other_count = other_count["Gender"]
other_percent = other_count/player_count
other_percent = "{:.2%}".format(other_percent)


In [228]:
#Purchasing Analysis (Gender)
#The below each broken by gender
gender_grouped_df = purchase_data_df.copy().groupby(["Gender"])
#Purchase Count
purchase_by_gender = gender_grouped_df["Purchase ID"].count()

#Average Purchase Price
average_by_gender = gender_grouped_df["Price"].mean()

#Total Purchase Value
total_by_gender = gender_grouped_df["Price"].sum()

#Average Purchase Total per Person by Gender
average_per_person_by_gender = 

#Summary DF
gender_purcahse_summary = pd.DataFrame({"Number of Purchases": purchase_by_gender,"Average Purchase": average_by_gender, "Total Purchase": total_by_gender, "Average Per Person by Gender": average_per_person_by_gender})
gender_purcahse_summary

,Number of Purchases,Average Purchase,Total Purchase,Average Per Person by Gender
Gender,,,,
Female,113,3.203009,361.94,0.008850
Male,652,3.017853,1967.64,0.001534
Other / Non-Disclosed,15,3.346000,50.19,0.066667


In [298]:
#Top Spenders
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

#SN
top_5 = purchase_data_df.copy()[["SN", "Price"]]
top_5 = top_5.groupby(["SN"]).sum().nlargest(5, ["Price"])

#Purchase Count
top_5_num_purchase = purchase_data_df.copy()[["SN", "Price"]]
top_5_num_purchase = top_5_num_purchase.groupby(["SN"]).count().sort_values("Price", ascending=False).head(5)

#Average Purchase Price
top_5_ave_purch = purchase_data_df.copy()[["SN", "Price"]]
top_5_ave_purch = top_5_ave_purch.groupby(["SN"]).mean().sort_values("Price", ascending=False).head(5)
#Total Purchase Value
top_5_total = purchase_data_df.copy()[["SN", "Price"]]
top_5_total = top_5_total.groupby(["SN"]).sum().sort_values("Price", ascending=False).head(5)



top_5_purch_summary = pd.DataFrame({"Top 5 Users": top_5, "Puchase Count": top_5_num_purchase, "Average Purchase Price": top_5_ave_purch, "Total Purchase Value for Each": top_5_total}, index=[0])
top_5_purch_summary

,Top 5 Users,Puchase Count,Average Purchase Price,Total Purchase Value for Each
0,"(P, r, i, c, e)","(P, r, i, c, e)","(P, r, i, c, e)","(P, r, i, c, e)"


In [203]:
#Print Analysis
print("")
print(f"Heroes of Pymoli Analysis")
print(f"--------------------------")
print(f"Player Count")
print(f"Total Number of Players: {player_count}")
print(f"")
print(f"Purchasing Analysis (Total)")
print(f"Number of Unique Items: {number_items}")
print("Average Purchase Price: ${:,.3}".format(average_purchase))
print(f"Total Number of Purchases: {total_num_purchases}")
print(f"Total Revenue: ${total_rev}")
print(f"")
print(f"Gender Demographics")
print("Percentage and Count of Male Players: {} players identify as Male, {} of all players".format(male_count, male_percent))
print("Percentage and Count of Female Players: {} players identify as Female, {} of all players".format(female_count, female_percent))
print("Percentage and Count of Other / Non-Disclosed: {} players identify as Other or Non-Disclose, {} of all players".format(other_count, other_percent))
print("")
print("{}".format(gender_purcahse_summary))
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")
print("")


Heroes of Pymoli Analysis
--------------------------
Player Count
Total Number of Players: 576

Purchasing Analysis (Total)
Number of Unique Items: 179
Average Purchase Price: $3.05
Total Number of Purchases: 780
Total Revenue: $2379.77

Gender Demographics
Percentage and Count of Male Players: 652 players identify as Male, 113.19% of all players
Percentage and Count of Female Players: 113 players identify as Female, 19.62% of all players
Percentage and Count of Other / Non-Disclosed: 15 players identify as Other or Non-Disclose, 2.60% of all players

                       Number of Purchases  Average Purchase  Total Purchase  \
Gender                                                                         
Female                                 113          3.203009          361.94   
Male                                   652          3.017853         1967.64   
Other / Non-Disclosed                   15          3.346000           50.19   

                       Average Per Perso

In [204]:
#create txt. file with analysis
f = open("Heroes of Pymoli Analysis.txt", "w+")
f.write(" "+ '\n')
f.write("Heroes of Pymoli Analysis"+ '\n')
f.write("--------------------------"+ '\n')
f.write("Player Count"+ "/n")
f.write(f"Total Number of Players: {player_count}"+ '\n')
f.write(" " + "/n")
f.write("Purchasing Analysis (Total)"+ '\n')
f.write(f"Number of Unique Items: {number_items}"+ '\n')
f.write("Average Purchase Price: ${:,.3}".format(average_purchase) + '\n')
f.write(f"Total Number of Purchases: {total_num_purchases}"+ '\n')
f.write(f"Total Revenue: ${total_rev}"+ '\n')
f.write(f" " + '\n')
f.write(f"Gender Demographics"+ '\n')
f.write("Percentage and Count of Male Players: {} players idendify as Male, {} of all players".format(male_count, male_percent) + '\n')
f.write("Percentage and Count of Female Players: {} players idendify as Female, {} of all players".format(female_count, female_percent) + '\n')
f.write("Percentage and Count of Other / Non-Disclosed: {} players idendify as Other or Non-Disclose, {} of all players".format(other_count, other_percent) + '\n')
f.write(f" " + '\n')
f.write(f"Purchasing Analysis (Gender)" + '\n')
f.write("{}".format(gender_purcahse_summary))
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')
f.write(f"" + '\n')




1